# Collision check

tensorflow estimator model을 사용할때 feature에 tf.feature_column module이 들어가야 한다.

보통 embedding(hashing(data))를 사용하는데, data의 unique count가 매우 큰 경우 이보다 작은 값으로 bucketsize를 정하기에 coliision이 발생할 수 있다.

In [9]:
import sys
import tensorflow as tf
# Fix seed
tf.random.set_seed(825)

data = {'keywords': ['1111', '2222', '3333', '4444']}

# A utility method to show transromation from feature column
def demo(feature_column, data):
    feature_layer = tf.keras.layers.DenseFeatures(feature_column)
    return feature_layer(data)

def emb_hash(bucket_size):
    keywords = tf.feature_column.categorical_column_with_hash_bucket("keywords",
    bucket_size)
    emb = tf.feature_column.embedding_column(keywords,2)
    return emb

# bucket size = 1
print(demo(emb_hash(1), data))
# all output is same

# bucket size = 2
print(demo(emb_hash(2), data))
# '1111' '4444' / '2222' '3333'

# bucket size = 3
print(demo(emb_hash(3), data))
# '1111' '2222' '3333' / '4444'

# bucket size = 4
print(demo(emb_hash(4), data))
# '1111' '4444' / '2222' '3333'

# bucket size = 8
print(demo(emb_hash(8), data))
# '1111' '4444' / '2222' / '3333'

# bucket size = 16
print(demo(emb_hash(16), data))
# all output is not same.

tf.Tensor(
[[-0.30677307 -1.0190516 ]
 [-0.30677307 -1.0190516 ]
 [-0.30677307 -1.0190516 ]
 [-0.30677307 -1.0190516 ]], shape=(4, 2), dtype=float32)
tf.Tensor(
[[ 0.04094386 -0.17662054]
 [-0.26557332 -0.2745159 ]
 [-0.26557332 -0.2745159 ]
 [ 0.04094386 -0.17662054]], shape=(4, 2), dtype=float32)
tf.Tensor(
[[-0.18865938  0.0809349 ]
 [ 0.6760714   0.09676189]
 [ 0.6760714   0.09676189]
 [-0.18865938  0.0809349 ]], shape=(4, 2), dtype=float32)
tf.Tensor(
[[-0.5353973   0.63823575]
 [-0.12666304 -0.14882074]
 [ 0.48956296  0.4375769 ]
 [-0.5353973   0.63823575]], shape=(4, 2), dtype=float32)
tf.Tensor(
[[-0.04169198 -1.2398397 ]
 [ 0.25786075  0.73310757]
 [ 0.9352041  -0.0208841 ]
 [-0.43077955 -0.31218588]], shape=(4, 2), dtype=float32)


이러한 collision을 막기 위한 방법으로 bloom embedding이라는 것이 있다.

bloom embedding이란 동일한 데이터에 대해 blooming(복제)를 해서 여러 feature를 만들고

이들을 다른 bucket size로 hashing한 다음 embedding하여 add 또는 concat함으로써 중복을 최소화하는 방식이다.

위의 경우 '1111' '4444'가 동일하고 '2222' '3333'이 동일한 bucket size 2인 경우와

'1111' '2222' '3333'이 동일하고 '4444' 혼자 다른 bucket size 3인 경우를 더하면

아래와 같이 '1111' / '2222' '3333' / '4444' 끼리 동일한 값을 갖게 되고 collision이 줄어들었음을 확인할 수 있다.

In [15]:
# bucket size = 2
bucket2 = demo(emb_hash(2), data)
print(f'bucket2 = {bucket2}')
# '1111' '4444' / '2222' '3333'

# bucket size = 3
bucket3 = demo(emb_hash(3), data)
print(f'\nbucket3 = {bucket3}')
# '1111' '2222' '3333' / '4444'


print(f'\nAdd bucket2 + bucket3 = {bucket2 + bucket4}')
# '1111' '4444' / '2222' '3333'

bucket2 = [[-0.7452543  -0.01533942]
 [ 0.46846437  0.15328027]
 [ 0.46846437  0.15328027]
 [-0.7452543  -0.01533942]]

bucket3 = [[-1.09138    -0.28673676]
 [-1.09138    -0.28673676]
 [-1.09138    -0.28673676]
 [-0.01979477 -0.3555715 ]]

Add bucket2 + bucket3 = [[-0.5869992   0.56720984]
 [ 0.6267195   0.73582953]
 [ 0.6267195   0.73582953]
 [-0.43409187 -1.3507265 ]]
